In [27]:
import cvxpy
from matplotlib import pyplot as plt
import numpy as np
from src.components.microgrid_simple import SimpleMicrogrid
from src.components.battery import Battery, BatteryParameters
from src.rl.a2c.c_mg_simple import set_all_seeds

### Set env vars

In [33]:
set_all_seeds(0)
n = 24

mg = SimpleMicrogrid(steps=n, min_temp=29, max_temp=31, peak_pv_gen=1, peak_grid_gen=1, peak_load=1, disable_noise=True)
demand = mg.demand
pv = mg.pv_gen
_, _, _, price_s, emission = mg.grid_price_and_emission()

price_b = price_s / 4

In [37]:
demand.sum(), pv,mg.price.sum(), mg.emission.sum()

(11.541017638708817, (24,), 2.6799999999999997, 3.6719999999999997)

In [35]:
battery_params = {
    "soc_0": 0.1,
    "soc_max":0.9,
    "soc_min":0.1,
    "p_charge_max":0.8,
    "p_discharge_max":0.8,
    "efficiency":0.9,
    "capacity":1,
    "sell_price":0.0,
    "buy_price":0.0
}

real_battery = Battery(random_soc_0=False, params = BatteryParameters(battery_params))

p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=0.8)
real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

real_battery.soc.item()

# battery.reset()

0.8200000000000001

### Adv battery calc (not working)

In [36]:
set_all_seeds(0)
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't
for i in range(n):
    constraints.append(action[i] <= 0.8)
    constraints.append(action[i] >= -0.8)

# Transition
obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery SOC
    # self.soc = self.soc + (p_charge * self.efficiency - p_discharge / self.efficiency) / self.capacity

    constraints.append(battery[i+1] == battery[i] + ((action[i] * battery_params["efficiency"])/battery_params["efficiency"])/battery_params["capacity"] )
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

(0.571068197307288,
 array([0.1       , 0.34058091, 0.4441487 , 0.51103194, 0.58053436,
        0.69231635, 0.9       , 0.8086571 , 0.1       , 0.28634816,
        0.89236155, 0.61229263, 0.51291948, 0.42732639, 0.21550249,
        0.80113181, 0.9       , 0.69301569, 0.20327912, 0.37999039,
        0.41320395, 0.37181947, 0.32199775, 0.2439456 , 0.1       ]),
 array([ 0.24058091,  0.10356779,  0.06688324,  0.06950242,  0.11178199,
         0.20768365, -0.0913429 , -0.7086571 ,  0.18634816,  0.60601339,
        -0.28006892, -0.09937314, -0.08559309, -0.2118239 ,  0.58562932,
         0.09886819, -0.20698431, -0.48973657,  0.17671127,  0.03321356,
        -0.04138449, -0.04982172, -0.07805214, -0.1439456 ]))

### Real dataset

In [33]:
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)


# Transition
obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery
    constraints.append(battery[i+1] == battery[i] + action[i] * battery_params["efficiency"])
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

(0.8576216440394087,
 array([0.1       , 0.35374129, 0.45503172, 0.50668433, 0.5582184 ,
        0.66026749, 0.9       , 0.86564381, 0.22729823, 0.1       ,
        0.23310394, 0.3611549 , 0.50728754, 0.66157729, 0.77398929,
        0.9       , 0.70576379, 0.2677359 , 0.1       , 0.34314748,
        0.42989392, 0.44876564, 0.4452682 , 0.32108506, 0.1       ]),
 array([ 0.28193477,  0.11254492,  0.0573918 ,  0.05726008,  0.11338788,
         0.26636945, -0.03817355, -0.70927286, -0.14144247,  0.14789326,
         0.14227884,  0.1623696 ,  0.17143305,  0.12490222,  0.1400119 ,
        -0.21581801, -0.48669766, -0.18637322,  0.27016387,  0.09638493,
         0.02096857, -0.00388604, -0.13798126, -0.24565006]))

### Test Dataset

In [ ]:
n = 4
d = np.random.rand(n)
g = np.random.rand(n)*0.5
pb = np.random.rand(n)
ps = pb/4

In [ ]:

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)


constraints = []
for i in range(n+1):
    constraints.append(battery[i] <= 1)
    constraints.append(battery[i] >= 0)
constraints.append(battery[0] == 0)

for i in range(n):
    constraints.append(action[i] <= .3)
    constraints.append(action[i] >= -1)
    

obj = 0
for i in range(n):
    constraints.append(battery[i+1] == battery[i] + action[i]  )

    constraints.append(consumption[i] == d[i]-g[i] + action[i] )


    obj += cvxpy.maximum(consumption[i]* pb[i],0) 
    obj += cvxpy.maximum(-consumption[i]*ps[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

In [ ]:


x_hat = cvxpy.Variable(A.shape[1])
objective = cvxpy.Minimize(cvxpy.norm(x_hat, 1))
    constraints = [A * x_hat == y]
    prob = cvxpy.Problem(objective, constraints)
    prob.solve()
    return np.squeeze(np.array(x_hat.value)), (prob.value, prob.status)